In [1]:
import torch.nn as nn
import torch

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
from skimage.filters import threshold_local, gaussian

In [2]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/047-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc"
# run_path = "/home/harishbabu/projects/PIPNet/runs/048-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc_unit-sphere"
# run_path = "/home/harishbabu/projects/PIPNet/runs/051-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/052-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/055-CUB-18_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/056-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/057-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/058-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"

# with unit sphere
# run_path = "/home/harishbabu/projects/PIPNet/runs/059-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# unit sphere with softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/065-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# original hpipnet with 20 protos per node no KO, no OOD, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/062-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

# original hpipnet with 20 protos per node no KO, no OOD, WITH tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/063-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD_tanh-desc"

# with unit sphere but no AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/066-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_no-align_no-uni_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc, INCORRECT
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-incorrect-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, no softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/068-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# 071 with bias
# run_path = "/home/harishbabu/projects/PIPNet/runs/071-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_with-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 072 gumbel softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/072-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 073 gumbel softmax, tau-1.0
# run_path = "/home/harishbabu/projects/PIPNet/runs/073-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax-tau=1_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 075 068 with focal loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/075-068-with-focal_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 076 cs followed by softmax. Uses align_pf along with align+uni
# run_path = "/home/harishbabu/projects/PIPNet/runs/076_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2-APW=5_batch=20"

# 074 multiply_cs_softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/074-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_multi-cs-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 077 unit sphere protopool with cosin no softmax constant 20 protos per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/077_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 082 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/082-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 083 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/083-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-align_no-uni_no-mm-loss_batch=48"

# 085 unit sphere cs followed by softmax-with-tau with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/085-notebook-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=12"

# 091 basic gaussian multiplier on stage 4
run_path = "/home/harishbabu/projects/PIPNet/runs/091-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 092 basic gaussian multiplier on stage 3, 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/092-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=3,4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 093 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/093-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 094 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/094-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 095 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/095-091-woALUNI-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-AL_no-UNI_mm-loss_batch=48"

# 096 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/096-091-wfocal-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 097 - 091 with bg
# run_path = "/home/harishbabu/projects/PIPNet/runs/097-091-wbg-CUB-18_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0100 cub29 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/100_CUB-29-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0101 baseline with 4 per desc per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/101-baseline-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_no-KO_no-OOD"

# 0103 091 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/103-091-wProtoPool20PerNode_CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48"

try:
    sys.path.remove('/home/harishbabu/projects/PIPNet')
except:
    pass
sys.path.insert(0, os.path.join(run_path, 'source_clone'))


In [3]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

In [4]:
from PIL import Image
import numpy as np
import pdb

def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

def get_bb_gaussian_threshold(latent_activation, sigma=1.0, percentile=97, extend_h=0, extend_w=0):
    # latent_activation -> []
    upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, \
                                                                 image_size=(args.image_size, args.image_size))
    upscaled_similarity = minmaxscale(upscaled_similarity)
    upscaled_similarity = gaussian(upscaled_similarity, sigma=sigma)
    upscaled_similarity = threshold_local(upscaled_similarity, block_size=15, method='mean')
    h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity ,percentile=97)
    h_min = max(0, h_min-extend_h)
    h_max = min(upscaled_similarity.shape[0], h_max+extend_h)
    w_min = max(0, w_min-extend_w)
    w_max = min(upscaled_similarity.shape[1], w_max+extend_w)
    return h_min, h_max, w_min, w_max


def minmaxscale(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())

from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

## Load Model

In [5]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

exp_no = int(os.path.basename(run_path)[:3])

if exp_no < 77:
    if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
        for node in root.nodes_with_children():
            node.set_num_protos(args.num_protos_per_descendant)
else:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))

------------------------- No discretization -------------------------


In [6]:
args.batch_size = 1

print(args.batch_size)

1


In [7]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

# args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
# args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)

print(args.batch_size, trainloader.batch_size)

if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
print(net.eval())
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
# with torch.no_grad():
#     xs1, _, _ = next(iter(trainloader))
#     xs1 = xs1.to(device)
#     proto_features, _, _ = net(xs1)
#     wshape = proto_features['root'].shape[-1]
#     args.wshape = wshape #needed for calculating image patch size
#     print("Output shape: ", proto_features['root'].shape, flush=True)
    
args.wshape = 26

Dropping 0 samples from trainloader
Dropping 0 samples from trainloader_normal
Dropping 0 samples from trainloader_normal_augment
Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
1 1
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}
stage 4
Number of prototypes:  768
----------Prototypes per descendant: 4----------
Assigned 72 protos to node root
Assig

In [8]:
args.basic_cnext_gaussian_multiplier

'4|1.0|50'

# Proto activations on leaf descendents - topk images using either NAIVE-HPIPNET or UNIT-SPACE-PROTOPOOL with HEATMAP (CANON)

In [10]:
# Proto activations on leaf descendents - topk images

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision
import torch.nn.functional as F



def find_top_percentile_bbox(image, percentile=95):
    threshold = np.percentile(image.flatten(), percentile)
    mask = image >= threshold
    coords = np.argwhere(mask)
    if coords.size == 0:
        return None, None, None, None
    h_min, w_min = coords.min(axis=0)
    h_max, w_max = coords.max(axis=0)
    h_min, h_max, w_min, w_max = map(int, [h_min, h_max, w_min, w_max])
    return h_min, h_max, w_min, w_max

def find_high_activation_crop(activation_map, percentile=95):
    threshold = np.percentile(activation_map, percentile)
    mask = np.ones(activation_map.shape)
    mask[activation_map < threshold] = 0
    lower_y, upper_y, lower_x, upper_x = 0, 0, 0, 0
    for i in range(mask.shape[0]):
        if np.amax(mask[i]) > 0.5:
            lower_y = i
            break
    for i in reversed(range(mask.shape[0])):
        if np.amax(mask[i]) > 0.5:
            upper_y = i
            break
    for j in range(mask.shape[1]):
        if np.amax(mask[:,j]) > 0.5:
            lower_x = j
            break
    for j in reversed(range(mask.shape[1])):
        if np.amax(mask[:,j]) > 0.5:
            upper_x = j
            break
    return lower_y, upper_y+1, lower_x, upper_x+1

def get_upscaled_activation_uninterpolated(latent_activation, image_size):
    image_a = latent_activation.cpu().numpy()
    min_image_a = image_a.min()
    max_image_a = image_a.max()
    image_a = (image_a - min_image_a) / (max_image_a - min_image_a)
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((image_size[-1], \
                                                                                            image_size[-2]), \
                                                                                          resample=Image.NEAREST ))
    
    reshaped_image_a = (reshaped_image_a / 255).astype('float16')
    reshaped_image_a = (reshaped_image_a * (max_image_a - min_image_a)) + min_image_a
    return reshaped_image_a

def functional_UnitConv2D(in_features, weight, bias, stride = 1, padding=0):
    normalized_weight = F.normalize(weight.data, p=2, dim=(1, 2, 3)) # Normalize the kernels to unit vectors
    normalized_input = F.normalize(in_features, p=2, dim=1) # Normalize the input to unit vectors
    if bias is not None:
        normalized_bias = F.normalize(bias.data, p=2, dim=0) # Normalize the kernels to unit vectors
    else:
        normalized_bias = None
    return F.conv2d(normalized_input, normalized_weight, normalized_bias, stride=stride, padding=padding)

def findCorrespondingToMax(base, target):
    output, indices = F.max_pool2d(base, kernel_size=(26, 26), return_indices=True)# these are logits
    tensor_flattened = target.view(target.shape[0], target.shape[1], -1)
    indices_flattened = indices.view(target.shape[0], target.shape[1], -1)
    corresponding_values_in_target = torch.gather(tensor_flattened, 2, indices_flattened)
    corresponding_values_in_target = corresponding_values_in_target.view(target.shape[0],\
                                     target.shape[1], 1, 1)
    pooled_target = corresponding_values_in_target
    return pooled_target

def customForwardWithCSandSoftmax(net, xs,  inference=False):
    features = net.module._net(xs) 
    proto_features = {}
    proto_features_cs = {}
    proto_features_softmaxed = {}
    pooled = {}
    pooled_cs = {}
    pooled_softmaxed = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        # this may or may not be cosine similarity based on UniConv2D or Conv2d
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)
        
        #calculating cosine similarity
        prototypes = getattr(net.module, '_'+node.name+'_add_on')
        proto_features_cs[node.name] = functional_UnitConv2D(features, prototypes.weight, prototypes.bias)

        if net.module.args.softmax == 'y':
            softmax_tau = 0.2
            proto_features[node.name] = proto_features[node.name] / softmax_tau
            proto_features_softmaxed[node.name] = net.module._softmax(proto_features[node.name])
            proto_features[node.name] = proto_features_softmaxed[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'
        elif net.module.args.gumbel_softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._gumbel_softmax(proto_features[node.name])
            proto_features[node.name] = proto_features_softmaxed[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'

        if net.module.args.multiply_cs_softmax == 'y':
            proto_features[node.name] = proto_features_cs[node.name] * proto_features_softmaxed[node.name]
        pooled[node.name] = net.module._pool(proto_features[node.name])
        
        # this could be softmax or cosine similarity
        pooled_cs[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_cs[node.name])
        
        pooled_softmaxed[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_softmaxed[node.name])

        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes) # these are logits

    return features, proto_features, pooled, pooled_cs, pooled_softmaxed, out

find_non_descendants = False # True, False # param
vizloader_name = 'testloader'
bbox_percentile = 97
topk = 10 # param, args param
save_images = True #True

font = ImageFont.truetype("arial.ttf", 50)
font2 = ImageFont.truetype("arial.ttf", 20)

from datetime import datetime
txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
txt_file.write('\n')

def write_num_proto_details(proto_mean_activations, node_name, net, threshold, txt_file, args):
    
    rand_input = torch.randn((1, 3, args.image_size, args.image_size))
    with torch.no_grad():
        *_, pooled, out = net(rand_input)
    num_protos = pooled[node_name].shape[1]
    used_protos = len(proto_mean_activations)
    non_overspecific = 0
    for p in proto_mean_activations:
        logstr = '\t'*2 + f'Proto:{p} '
        protos_mean_for_all_leaf_descedants = []
        for leaf_descendent in proto_mean_activations[p]:
            mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
            protos_mean_for_all_leaf_descedants.append(mean_activation)
            
        if all([(mean_activation>0.2) for mean_activation in protos_mean_for_all_leaf_descedants]):
            non_overspecific += 1
            
    txt_file.write(f"Node:{node_name},Total:{num_protos},Used:{used_protos},Good:{non_overspecific},threshold={threshold}\n")


def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)



vizloader_dict = {'trainloader': trainloader,
                 'projectloader': projectloader,
                 'testloader': testloader,
                 'test_projectloader': test_projectloader}
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])


if type(vizloader_dict[vizloader_name].dataset) == ImageFolder:
    name2label = vizloader_dict[vizloader_name].dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
else:
    name2label = vizloader_dict[vizloader_name].dataset.dataset.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

#     name2label = projectloader.dataset.class_to_idx # param
#     label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(vizloader_dict[vizloader_name], node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        # change
#         if not find_non_descendants: 
#             # do only when finding descendants
#             if coarse_label2name[ys.item()] not in non_leaf_children_names:
#                 continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            model_output = customForwardWithCSandSoftmax(net, xs, inference=False)
            _, softmaxes, pooled, pooled_ip, pooled_softmax, _ = model_output
#             model_output = net(xs, inference=False)
#             if len(model_output) == 3:
#                 softmaxes, pooled, _ = model_output
#             elif len(model_output) == 4:
#                 _, softmaxes, pooled, _ = model_output
            pooled = pooled[node.name].squeeze(0)
            pooled_ip = pooled_ip[node.name].squeeze(0) 
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                # change
#                 if (len(relevant_proto_class_names) == 1):# and (relevant_proto_class_names[0] not in non_leaf_children_names):
#                     continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                if not find_non_descendants:
                    if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                else:
                    if (coarse_label2name[ys.item()] not in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)
                
    write_num_proto_details(proto_mean_activations, node.name, net, threshold=0.2, txt_file=txt_file, args=args)
    
    print('Node', node.name)
    for class_label in range(classification_weights.shape[0]):
        child_name = (coarse_label2name[class_label])
        print('Num protos for', child_name, torch.nonzero(classification_weights[class_label, :] > 1e-3).shape[0])
        
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            
            logstr = '\t'*2 + f'Proto:{p} '
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
            print(logstr)
            
            # have this for NON descendants
            if len(proto_mean_activations[p]) == 0:
                continue
            
            if save_images:
                patches = []
                right_descriptions = []
                text_region_width = 3 # 3x the width of a patch
                for leaf_descendent, heap in proto_mean_activations[p].items():
                    heap = sorted(heap)[::-1]
                    mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    least_activation = min([round(activation, 2) for activation, *_ in proto_mean_activations[p][leaf_descendent]])
                    most_activation = max([round(activation, 2) for activation, *_ in proto_mean_activations[p][leaf_descendent]])
                    mean_cosine_similarity = round(np.mean([activation_inner_product for _, activation_inner_product, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    for ele in heap:
                        activation, activation_inner_product, img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation = ele
                        image = transforms.Resize(size=(args.image_size, args.image_size))(Image.open(img_to_open))
                        img_tensor = transforms.ToTensor()(image)#.unsqueeze_(0) #shape (1, 3, h, w)
                        img_tensor_patch = img_tensor[:, h_coor_min:h_coor_max, w_coor_min:w_coor_max]
#                         overlayed_image_np = get_heatmap(latent_activation, img_tensor)
#                         overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
#                         patches.append(overlayed_image)
                        
                        overlayed_image_np = get_heatmap(latent_activation, img_tensor)
                        
                        overlayed_image_pil = Image.fromarray(overlayed_image_np)
                        draw = D.Draw(overlayed_image_pil)
                        text = f"{round(activation, 2), round(activation_inner_product, 2)}"
#                         text_width, text_height = draw.textsize(text, font2)
                        bbox = draw.textbbox((0, 0), text, font2)
                        text_width = bbox[2] - bbox[0]
                        text_height = bbox[3] - bbox[1]
                        x, y = 224 - text_width - 5, 5  # 10 pixels padding from right
                        draw.text((x, y), text, font=font2, fill=(255, 255, 255))
                        overlayed_image_np = np.array(overlayed_image_pil)
                        
                        overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
                        
#                         pdb.set_trace()
                        upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, image_size=(args.image_size, args.image_size))
#                         h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity, percentile=bbox_percentile)
                        h_min, h_max, w_min, w_max = get_bb_gaussian_threshold(latent_activation, sigma=1.0, \
                                                                               percentile=bbox_percentile, extend_h=0, extend_w=0)

        #                         h_min, h_max, w_min, w_max = find_high_activation_crop(upscaled_similarity)
#                         h_min, h_max, w_min, w_max
                        bbox_coords = torch.tensor([[w_min, h_min, w_max, h_max]])
                        overlayed_bb_image = torchvision.utils.draw_bounding_boxes((overlayed_image * 255).type(torch.uint8), \
                                                                                   bbox_coords, colors='red') / 255
                        
#                         plt_image = overlayed_bb_image.permute(1, 2, 0)# should be H, W, C with 0 to 1
#                         plt.imshow(plt_image)
#                         plt.show()
#                         pdb.set_trace()
                        patches.append(overlayed_bb_image)

                    # description on the right hand side
                    text = f'{mean_activation}, {leaf_descendent}'
                    txtimage = Image.new("RGB", (patches[0].shape[-2]*text_region_width,patches[0].shape[-1]), (0, 0, 0))
                    draw = D.Draw(txtimage)
                    draw.text((200, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                    txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)
                    right_descriptions.append(txttensor)
                
                # weird thing padding should be zero for non descendants else it raises some error # change
                if find_non_descendants or (len(patches) == topk): # (len(patches) == topk) means there is only one leaf descendant
                    padding = 0
                else:
                    padding = 1

                grid = torchvision.utils.make_grid(patches, nrow=topk, padding=padding)
                grid_right_descriptions = torchvision.utils.make_grid(right_descriptions, nrow=1, padding=padding)

                # merging right description with the grid of images
                grid = torch.cat([grid, grid_right_descriptions], dim=-1)

                # description on the top
                text = f'Node:{node.name}, p{p}, Child:{child_classname}'
                txtimage = Image.new("RGB", (grid.shape[-1], 224), (0, 0, 0))
                draw = D.Draw(txtimage)
                draw.text((350, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)

                # merging top description with the grid of images
                grid = torch.cat([grid, txttensor], dim=1)
                
                prefix = 'non_' if find_non_descendants else ''
                os.makedirs(os.path.join(run_path, prefix + f'descendent_specific_topk_heatmap_{vizloader_name}_{bbox_percentile}_ep={epoch}', node.name), exist_ok=True)
                torchvision.utils.save_image(grid, os.path.join(run_path, prefix + f'descendent_specific_topk_heatmap_{vizloader_name}_{bbox_percentile}_ep={epoch}', node.name, f'{child_classname}-p{p}.png'))

txt_file.write('\n')
txt_file.close()
print('Done !!!')

/tmp/ipykernel_120018/2075859963.py:56: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST ))


Node root
Num protos for 052+053 6
Num protos for 004+086 9
	 Child: 052+053
		Proto:4 050:(0.1584) 051:(0.1613) 052:(0.0262) 053:(0.1531) 
		Proto:6 050:(0.3389) 051:(0.3602) 052:(0.0438) 053:(0.2627) 
		Proto:7 050:(0.0697) 051:(0.0763) 052:(0.1262) 053:(0.0583) 
		Proto:11 050:(0.5929) 051:(0.5928) 052:(0.0729) 053:(0.5718) 
		Proto:12 050:(0.2442) 051:(0.2369) 052:(0.2751) 053:(0.1843) 
		Proto:14 050:(0.3259) 051:(0.3583) 052:(0.3991) 053:(0.2532) 
	 Child: 004+086
		Proto:64 001:(0.0561) 002:(0.0565) 003:(0.0588) 004:(0.0557) 023:(0.0552) 024:(0.0521) 025:(0.0554) 031:(0.0575) 032:(0.0579) 033:(0.0576) 045:(0.0552) 086:(0.0489) 100:(0.0528) 101:(0.0463) 
		Proto:33 001:(0.5691) 002:(0.5804) 003:(0.5647) 004:(0.5839) 023:(0.5697) 024:(0.5438) 025:(0.5748) 031:(0.5878) 032:(0.5854) 033:(0.5927) 045:(0.5283) 086:(0.1636) 100:(0.5864) 101:(0.5765) 
		Proto:65 001:(0.157) 002:(0.1486) 003:(0.1565) 004:(0.1482) 023:(0.1312) 024:(0.1169) 025:(0.1219) 031:(0.1564) 032:(0.1552) 033:(0.152

Node 052+053
Num protos for cub_052_Pied_billed_Grebe 3
Num protos for 053+050 3
	 Child: cub_052_Pied_billed_Grebe
		Proto:0 052:(0.7463) 
		Proto:2 052:(0.8737) 
		Proto:3 052:(0.8712) 
	 Child: 053+050
		Proto:10 050:(0.848) 051:(0.8516) 053:(0.8628) 
		Proto:12 050:(0.8507) 051:(0.8482) 053:(0.8432) 
		Proto:7 050:(0.7396) 051:(0.7222) 053:(0.7762) 


Node 004+086
Num protos for 004+032 7
Num protos for 086+045 4
	 Child: 004+032
		Proto:1 004:(0.1332) 031:(0.234) 032:(0.2375) 033:(0.2727) 
		Proto:5 004:(0.0299) 031:(0.0406) 032:(0.0454) 033:(0.0437) 
		Proto:8 004:(0.0336) 031:(0.0511) 032:(0.0428) 033:(0.0424) 
		Proto:9 004:(0.0638) 031:(0.0643) 032:(0.0645) 033:(0.0641) 
		Proto:11 004:(0.5983) 031:(0.6442) 032:(0.6329) 033:(0.645) 
		Proto:12 004:(0.38) 031:(0.545) 032:(0.532) 033:(0.5496) 
		Proto:14 004:(0.0888) 031:(0.1065) 032:(0.0982) 033:(0.1062) 
	 Child: 086+045
		Proto:32 001:(0.5816) 002:(0.4491) 003:(0.5736) 023:(0.5532) 024:(0.2877) 025:(0.275) 045:(0.5539) 086:(0.5738) 100:(0.4556) 101:(0.4808) 
		Proto:34 001:(0.2825) 002:(0.2608) 003:(0.2792) 023:(0.255) 024:(0.0927) 025:(0.1839) 045:(0.3049) 086:(0.2939) 100:(0.3102) 101:(0.3158) 
		Proto:43 001:(0.5909) 002:(0.542) 003:(0.5842) 023:(0.5783) 024:(0.5681) 025:(0.6024) 045:(0.4989) 086:(0.5469) 100:(0.5731) 101:(0.4789) 
		Proto:23 001:(0.674) 002:(0.6866) 003:(0

Node 053+050
Num protos for cub_053_Western_Grebe 1
Num protos for 050+051 2
	 Child: cub_053_Western_Grebe
		Proto:2 053:(0.9284) 
	 Child: 050+051
		Proto:8 050:(0.9135) 051:(0.9154) 
		Proto:6 050:(0.9156) 051:(0.9147) 


...

Node 045+101
Num protos for 045+003 2
Num protos for 101+023 4
	 Child: 045+003
		Proto:1 001:(0.7687) 002:(0.7784) 003:(0.7745) 045:(0.7765) 
		Proto:13 001:(0.7758) 002:(0.7647) 003:(0.7753) 045:(0.7729) 
	 Child: 101+023
		Proto:18 023:(0.0758) 024:(0.0585) 025:(0.0654) 100:(0.1223) 101:(0.0621) 
		Proto:27 023:(0.5601) 024:(0.5621) 025:(0.5708) 100:(0.4226) 101:(0.0531) 
		Proto:28 023:(0.0502) 024:(0.0555) 025:(0.0483) 100:(0.356) 101:(0.3698) 
		Proto:21 023:(0.7799) 024:(0.7789) 025:(0.7843) 100:(0.7789) 101:(0.7832) 


Node 045+003
Num protos for cub_045_Northern_Fulmar 2
Num protos for 003+002 4
	 Child: cub_045_Northern_Fulmar
		Proto:0 045:(0.7619) 
		Proto:3 045:(0.8704) 
	 Child: 003+002
		Proto:13 001:(0.2571) 002:(0.2379) 003:(0.2426) 
		Proto:12 001:(0.2995) 002:(0.2923) 003:(0.2857) 
		Proto:5 001:(0.8805) 002:(0.8797) 003:(0.8771) 
		Proto:7 001:(0.8827) 002:(0.8677) 003:(0.8811) 


Node 101+023
Num protos for 101+100 3
Num protos for 023+025 3
	 Child: 101+100
		Proto:0 100:(0.8696) 101:(0.8701) 
		Proto:2 100:(0.4593) 101:(0.3966) 
		Proto:6 100:(0.7539) 101:(0.7309) 
	 Child: 023+025
		Proto:9 023:(0.2051) 024:(0.1904) 025:(0.1809) 
		Proto:11 023:(0.7843) 024:(0.8424) 025:(0.8381) 
		Proto:12 023:(0.8572) 024:(0.8649) 025:(0.8635) 


Node 003+002
Num protos for cub_003_Sooty_Albatross 2
Num protos for 002+001 2
	 Child: cub_003_Sooty_Albatross
		Proto:0 003:(0.8966) 
		Proto:3 003:(0.9198) 
	 Child: 002+001
		Proto:10 001:(0.8998) 002:(0.8996) 
		Proto:11 001:(0.9261) 002:(0.9244) 


Node 023+025
Num protos for cub_023_Brandt_Cormorant 3
Num protos for 025+024 4
	 Child: cub_023_Brandt_Cormorant
		Proto:1 023:(0.8114) 
		Proto:2 023:(0.9222) 
		Proto:3 023:(0.7174) 
	 Child: 025+024
		Proto:10 024:(0.9072) 025:(0.8919) 
		Proto:11 024:(0.5142) 025:(0.6037) 
		Proto:6 024:(0.8985) 025:(0.786) 
		Proto:7 024:(0.8958) 025:(0.6866) 
Done !!!


In [16]:
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )

    return new_dataloader


vizloader_name = 'testloader'
vizloader_dict = {'trainloader': trainloader,
                 'projectloader': projectloader,
                 'testloader': testloader,
                 'test_projectloader': test_projectloader}

vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

print(vizloader_dict[vizloader_name].sampler)

print(vizloader_dict[vizloader_name].batch_sampler)

Dataset ImageFolder
    Number of datapoints: 844
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/test_crop
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [17]:
vizloader_name = 'testloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 844
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/test_crop
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [18]:
vizloader_name = 'test_projectloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 844
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/test_full
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [19]:
vizloader_name = 'projectloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 869
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [22]:
vizloader_name = 'trainloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 869
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/train_crop


# Proto activations on leaf descendents - topk images using  NAIVE-HPIPNET with HEATMAP

In [8]:
# Proto activations on leaf descendents - topk images

def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision
from datetime import datetime
txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
txt_file.write('\n')

find_non_descendants = True # True, False # param
topk = 10
save_images = False # True
font = ImageFont.truetype("arial.ttf", 50)

from datetime import datetime
txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
txt_file.write('\n')

def write_num_proto_details(proto_mean_activations, node_name, net, threshold, txt_file, args):
    
    rand_input = torch.randn((1, 3, args.image_size, args.image_size))
    with torch.no_grad():
        *_, pooled, out = net(rand_input)
    num_protos = pooled[node_name].shape[1]
    used_protos = len(proto_mean_activations)
    non_overspecific = 0
    for p in proto_mean_activations:
        logstr = '\t'*2 + f'Proto:{p} '
        protos_mean_for_all_leaf_descedants = []
        for leaf_descendent in proto_mean_activations[p]:
            mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
            protos_mean_for_all_leaf_descedants.append(mean_activation)
            
        if all([(mean_activation>0.2) for mean_activation in protos_mean_for_all_leaf_descedants]):
            non_overspecific += 1
            
    txt_file.write(f"Node:{node_name},Total:{num_protos},Used:{used_protos},Good:{non_overspecific},threshold={threshold}\n")


def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = projectloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(projectloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
#         if coarse_label2name[ys.item()] not in non_leaf_children_names:
#             continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            model_output = net(xs, inference=False)
            if len(model_output) == 3:
                softmaxes, pooled, _ = model_output
            elif len(model_output) == 4:
                _, softmaxes, pooled, _ = model_output
            pooled = pooled[node.name].squeeze(0) 
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
#                 if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
#                     continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                
                if not find_non_descendants:
                    if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                else:
                    if (coarse_label2name[ys.item()] not in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

    write_num_proto_details(proto_mean_activations, node.name, net, threshold=0.2, txt_file=txt_file, args=args)

    print('Node', node.name)
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            
            logstr = '\t'*2 + f'Proto:{p} '
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
            print(logstr)
            
            # have this for NON descendants
            if len(proto_mean_activations[p]) == 0:
                continue
            
            if save_images:
                patches = []
                right_descriptions = []
                text_region_width = 3 # 3x the width of a patch
                for leaf_descendent, heap in proto_mean_activations[p].items():
                    heap = sorted(heap)[::-1]
                    mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                    for ele in heap:
                        activation, img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation = ele
                        image = transforms.Resize(size=(args.image_size, args.image_size))(Image.open(img_to_open))
                        img_tensor = transforms.ToTensor()(image)#.unsqueeze_(0) #shape (1, 3, h, w)
                        img_tensor_patch = img_tensor[:, h_coor_min:h_coor_max, w_coor_min:w_coor_max]
                        overlayed_image_np = get_heatmap(latent_activation, img_tensor)
                        overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
                        patches.append(overlayed_image)

                    # description on the right hand side
                    text = f'{mean_activation}, {leaf_descendent}'
                    txtimage = Image.new("RGB", (patches[0].shape[-2]*text_region_width,patches[0].shape[-1]), (0, 0, 0))
                    draw = D.Draw(txtimage)
                    draw.text((150, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                    txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)
                    right_descriptions.append(txttensor)
                    
                # weird thing padding should be zero for non descendants else it raises some error # change
                if find_non_descendants or (len(patches) == topk): # (len(patches) == topk) means there is only one leaf descendant
                    padding = 0
                else:
                    padding = 1

                grid = torchvision.utils.make_grid(patches, nrow=topk, padding=padding)
                grid_right_descriptions = torchvision.utils.make_grid(right_descriptions, nrow=1, padding=padding)

                # merging right description with the grid of images
                grid = torch.cat([grid, grid_right_descriptions], dim=-1)

                # description on the top
                text = f'Node:{node.name}, p{p}, Child:{child_classname}'
                txtimage = Image.new("RGB", (grid.shape[-1], args.wshape), (0, 0, 0))
                draw = D.Draw(txtimage)
                draw.text((150, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)

                # merging top description with the grid of images
                grid = torch.cat([grid, txttensor], dim=1)
                
                prefix = 'non_' if find_non_descendants else ''
                os.makedirs(os.path.join(run_path, prefix+f'descendent_specific_topk_heatmap_ep={epoch}', node.name), exist_ok=True)
                torchvision.utils.save_image(grid, os.path.join(run_path, prefix+f'descendent_specific_topk_heatmap_ep={epoch}', node.name, f'{child_classname}-p{p}.png'))

txt_file.write('\n')
txt_file.close()
print('Done !!!')

Node root
	 Child: 004+086
		Proto:0 050:(0.1413) 051:(0.066) 052:(0.023) 053:(0.0916) 
		Proto:1 050:(0.1628) 051:(0.0936) 052:(0.0049) 053:(0.0121) 
		Proto:2 050:(0.4334) 051:(0.4459) 052:(0.5046) 053:(0.0778) 
		Proto:4 050:(0.0286) 051:(0.0665) 052:(0.0823) 053:(0.0239) 
		Proto:5 050:(0.1461) 051:(0.2236) 052:(0.4905) 053:(0.0085) 
		Proto:6 050:(0.152) 051:(0.007) 052:(0.2272) 053:(0.187) 
		Proto:7 050:(0.041) 051:(0.05) 052:(0.0056) 053:(0.0209) 
		Proto:12 050:(0.0169) 051:(0.0194) 052:(0.0286) 053:(0.0016) 
		Proto:13 050:(0.355) 051:(0.2916) 052:(0.4347) 053:(0.4544) 
		Proto:14 050:(0.1041) 051:(0.1035) 052:(0.4102) 053:(0.0712) 
		Proto:16 050:(0.0315) 051:(0.0042) 052:(0.0015) 053:(0.0055) 
		Proto:18 050:(0.0134) 051:(0.0024) 052:(0.0027) 053:(0.1834) 
	 Child: 052+053
		Proto:3 001:(0.0101) 002:(0.0662) 003:(0.1745) 004:(0.0263) 023:(0.2362) 024:(0.2267) 025:(0.3117) 031:(0.0352) 032:(0.1059) 033:(0.0515) 045:(0.283) 086:(0.2296) 100:(0.0271) 101:(0.2492) 
		Proto:8 00

Node 052+053
	 Child: cub_052_Pied_billed_Grebe
		Proto:0 050:(0.514) 051:(0.4658) 053:(0.0621) 
		Proto:8 050:(0.1686) 051:(0.2244) 053:(0.0871) 
		Proto:4 050:(0.9143) 051:(0.7282) 053:(0.5018) 
		Proto:14 050:(0.3465) 051:(0.2433) 053:(0.4938) 
	 Child: 053+050
		Proto:1 052:(0.097) 
		Proto:2 052:(0.0127) 
		Proto:3 052:(0.0228) 
		Proto:6 052:(0.0621) 
		Proto:7 052:(0.0014) 
		Proto:9 052:(0.0001) 
		Proto:10 052:(0.0433) 
		Proto:11 052:(0.0934) 
		Proto:12 052:(0.0751) 
		Proto:13 052:(0.9944) 
		Proto:16 052:(0.2382) 
		Proto:17 052:(0.0461) 
		Proto:19 052:(0.0004) 


Node 004+086
	 Child: 004+032
		Proto:0 001:(0.0022) 002:(0.0019) 003:(0.0107) 023:(0.0023) 024:(0.0055) 025:(0.0016) 045:(0.0017) 086:(0.003) 100:(0.0039) 101:(0.0201) 
		Proto:2 001:(0.0293) 002:(0.0361) 003:(0.1002) 023:(0.1326) 024:(0.1622) 025:(0.1323) 045:(0.0559) 086:(0.1346) 100:(0.0132) 101:(0.0229) 
		Proto:6 001:(0.0044) 002:(0.0046) 003:(0.0051) 023:(0.0025) 024:(0.002) 025:(0.0054) 045:(0.0055) 086:(0.1393) 100:(0.0005) 101:(0.0068) 
		Proto:9 001:(0.0173) 002:(0.0151) 003:(0.0148) 023:(0.0108) 024:(0.0152) 025:(0.0123) 045:(0.0212) 086:(0.0246) 100:(0.0101) 101:(0.0142) 
		Proto:12 001:(0.0336) 002:(0.062) 003:(0.0152) 023:(0.0193) 024:(0.0292) 025:(0.0306) 045:(0.0325) 086:(0.0282) 100:(0.006) 101:(0.0056) 
		Proto:13 001:(0.0026) 002:(0.001) 003:(0.0626) 023:(0.0089) 024:(0.0048) 025:(0.0022) 045:(0.0057) 086:(0.0524) 100:(0.0088) 101:(0.0973) 
		Proto:15 001:(0.0102) 002:(0.0637) 003:(0.1175) 023:(0.0943) 024:(0.0554) 025:(0.0117) 045:(0.0123) 086:(0.0038) 100:(0.0056)

Node 053+050
	 Child: 050+051
		Proto:2 053:(0.0008) 
		Proto:3 053:(0.0209) 
		Proto:5 053:(0.0011) 
		Proto:8 053:(0.021) 
		Proto:9 053:(0.0011) 
		Proto:12 053:(0.0034) 
		Proto:13 053:(0.0364) 
		Proto:15 053:(0.0016) 
		Proto:17 053:(0.0002) 
		Proto:19 053:(0.0038) 
	 Child: cub_053_Western_Grebe
		Proto:6 050:(0.7647) 051:(0.4627) 
		Proto:7 050:(0.3891) 051:(0.2601) 
		Proto:10 050:(0.7771) 051:(0.6118) 
		Proto:11 050:(0.5746) 051:(0.6137) 
		Proto:14 050:(0.1598) 051:(0.1228) 
		Proto:18 050:(0.369) 051:(0.4317) 


Node 004+032
	 Child: 032+033
		Proto:0 004:(0.0018) 
		Proto:3 004:(0.0261) 
		Proto:4 004:(0.002) 
		Proto:6 004:(0.0505) 
		Proto:7 004:(0.0005) 
		Proto:8 004:(0.0005) 
		Proto:9 004:(0.0097) 
		Proto:10 004:(0.0127) 
		Proto:12 004:(0.0034) 
		Proto:13 004:(0.0208) 
		Proto:14 004:(0.1054) 
		Proto:15 004:(0.1092) 
		Proto:17 004:(0.0069) 
		Proto:18 004:(0.0002) 
		Proto:19 004:(0.0012) 
	 Child: cub_004_Groove_billed_Ani
		Proto:16 031:(0.4222) 032:(0.398) 033:(0.2952) 
		Proto:11 031:(0.2787) 032:(0.1474) 033:(0.2138) 
		Proto:5 031:(0.0743) 032:(0.1514) 033:(0.1302) 


...

Node 045+101
	 Child: 101+023
		Proto:0 001:(0.0252) 002:(0.023) 003:(0.0028) 045:(0.0015) 
		Proto:1 001:(0.1762) 002:(0.0206) 003:(0.0029) 045:(0.0009) 
		Proto:2 001:(0.005) 002:(0.0097) 003:(0.0022) 045:(0.0014) 
		Proto:6 001:(0.1018) 002:(0.0095) 003:(0.0101) 045:(0.0003) 
		Proto:7 001:(0.0879) 002:(0.015) 003:(0.0072) 045:(0.0054) 
		Proto:8 001:(0.0691) 002:(0.0307) 003:(0.0067) 045:(0.0251) 
		Proto:11 001:(0.1149) 002:(0.3251) 003:(0.0222) 045:(0.2468) 
		Proto:19 001:(0.0407) 002:(0.0059) 003:(0.0071) 045:(0.021) 
	 Child: 045+003
		Proto:3 023:(0.0008) 024:(0.0011) 025:(0.0099) 100:(0.0104) 101:(0.0141) 
		Proto:4 023:(0.0202) 024:(0.0297) 025:(0.0989) 100:(0.0193) 101:(0.0711) 
		Proto:5 023:(0.0908) 024:(0.0077) 025:(0.0025) 100:(0.0834) 101:(0.015) 
		Proto:9 023:(0.1281) 024:(0.0806) 025:(0.2426) 100:(0.0284) 101:(0.2334) 
		Proto:10 023:(0.2117) 024:(0.0301) 025:(0.1724) 100:(0.0067) 101:(0.0237) 
		Proto:12 023:(0.0908) 024:(0.0183) 025:(0.19) 100:(0.0165) 101:(0.023

Node 045+003
	 Child: 003+002
		Proto:0 045:(0.1132) 
		Proto:1 045:(0.1485) 
		Proto:2 045:(0.1978) 
		Proto:3 045:(0.1121) 
		Proto:4 045:(0.0804) 
		Proto:9 045:(0.1946) 
		Proto:10 045:(0.5904) 
		Proto:11 045:(0.2353) 
		Proto:15 045:(0.1971) 
		Proto:16 045:(0.1193) 
		Proto:17 045:(0.1023) 
		Proto:18 045:(0.1481) 
	 Child: cub_045_Northern_Fulmar
		Proto:8 001:(0.5401) 002:(0.4671) 003:(0.4399) 
		Proto:14 001:(0.1095) 002:(0.1602) 003:(0.0445) 
		Proto:6 001:(0.2596) 002:(0.2824) 003:(0.5059) 
		Proto:7 001:(0.0672) 002:(0.216) 003:(0.3992) 


Node 101+023
	 Child: 023+025
		Proto:0 100:(0.028) 101:(0.1503) 
		Proto:2 100:(0.0026) 101:(0.012) 
		Proto:4 100:(0.0169) 101:(0.0082) 
		Proto:6 100:(0.004) 101:(0.004) 
		Proto:9 100:(0.0118) 101:(0.0789) 
		Proto:10 100:(0.0058) 101:(0.0387) 
		Proto:11 100:(0.0016) 101:(0.0597) 
		Proto:18 100:(0.0602) 101:(0.0184) 
	 Child: 101+100
		Proto:1 023:(0.512) 024:(0.3051) 025:(0.6193) 
		Proto:3 023:(0.0361) 024:(0.0205) 025:(0.0067) 
		Proto:7 023:(0.2089) 024:(0.0167) 025:(0.0097) 
		Proto:8 023:(0.0384) 024:(0.0226) 025:(0.0232) 
		Proto:13 023:(0.1166) 024:(0.0225) 025:(0.0248) 
		Proto:15 023:(0.1816) 024:(0.196) 025:(0.0948) 
		Proto:16 023:(0.0435) 024:(0.0466) 025:(0.0109) 


Node 003+002
	 Child: 002+001
		Proto:0 003:(0.114) 
		Proto:3 003:(0.2846) 
		Proto:9 003:(0.2073) 
		Proto:11 003:(0.1632) 
		Proto:12 003:(0.0236) 
		Proto:13 003:(0.65) 
		Proto:15 003:(0.4872) 
		Proto:17 003:(0.1341) 
		Proto:18 003:(0.0713) 
	 Child: cub_003_Sooty_Albatross
		Proto:8 001:(0.3567) 002:(0.4712) 
		Proto:1 001:(0.2348) 002:(0.2916) 
		Proto:5 001:(0.1291) 002:(0.3136) 
		Proto:7 001:(0.2955) 002:(0.2927) 


Node 023+025
	 Child: cub_023_Brandt_Cormorant
		Proto:0 024:(0.0384) 025:(0.3325) 
		Proto:4 024:(0.299) 025:(0.3311) 
		Proto:7 024:(0.9827) 025:(0.1947) 
		Proto:17 024:(0.3439) 025:(0.4684) 
		Proto:18 024:(0.1361) 025:(0.0737) 
	 Child: 025+024
		Proto:1 023:(0.2085) 
		Proto:3 023:(0.135) 
		Proto:5 023:(0.6228) 
		Proto:6 023:(0.0921) 
		Proto:10 023:(0.0853) 
		Proto:12 023:(0.0598) 
		Proto:13 023:(0.1021) 
		Proto:14 023:(0.1417) 
Done !!!
